## Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the weatherpy_database.csv file. 
file_to_load = "../weather_database/weatherpy_database.csv"
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,39.22,81,40,18.41,scattered clouds
1,1,Butaritari,KI,3.0707,172.7902,82.02,77,12,15.64,light rain
2,2,Bambous Virieux,MU,-20.3428,57.7575,73.65,73,40,3.44,scattered clouds
3,3,Huarmey,PE,-10.0681,-78.1522,65.97,69,25,5.84,scattered clouds
4,4,Yarada,IN,17.6500,83.2667,86.32,79,40,4.61,scattered clouds


In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.0707,172.7902,82.02,77,12,15.64,light rain
4,4,Yarada,IN,17.6500,83.2667,86.32,79,40,4.61,scattered clouds
5,5,Ponta Do Sol,PT,32.6667,-17.1000,78.01,69,40,2.17,scattered clouds
8,8,Atuona,PF,-9.8000,-139.0333,77.92,68,2,20.24,clear sky
9,9,Novosergiyevka,RU,52.0934,53.6528,76.53,62,97,7.61,overcast clouds
14,14,Hatillo,PR,18.4863,-66.8254,89.20,74,97,8.01,overcast clouds
18,18,Norsup,VU,-16.0667,167.3833,79.23,85,96,11.45,overcast clouds
19,19,Praia,CV,14.9215,-23.5087,86.54,42,20,14.97,few clouds
26,26,Albany,US,42.6001,-73.9662,81.28,52,4,12.28,clear sky
34,34,Boone,US,39.0334,-92.3335,83.39,62,100,2.80,overcast clouds


In [7]:
# Check for null values
preferred_cities_df.count()

City_ID                261
City                   261
Country                261
Lat                    261
Lng                    261
Max Temp               261
Humidity               261
Cloudiness             261
Wind Speed             261
Current Description    261
dtype: int64

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    

    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

In [ ]:
# 7. Drop the rows where there is no Hotel Name.


In [ ]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
